In [12]:
import numpy as np
import scipy.linalg as la
from scipy.sparse import spdiags
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d

**Exercises 1 and 2**

In [13]:
# Copy your interiorPoint() function from the previous lab into your 
# solutions file for this lab, renaming it qInteriorPoint().

def qInteriorPoint(Q, c, A, b, guess, niter=20, tol=1e-16):
    n = len(c)  # length of x, c, μ
    m = len(b)  # length of b, y
    
    def F(x, y, μ):
        M = μ * np.eye(m)
        Y = y * np.eye(m)
        e = np.ones(m)
        r1 = np.array(Q @ x - A.T @ μ + c)
        r2 = np.array(A @ x - y - b)
        r3 = np.array(Y @ M @ e)
        return np.concatenate((r1, r2, r3))
    
    σ = 1/10
    
    DFr1 = np.hstack((Q, np.zeros((n, m)), -A.T))
    DFr2 = np.hstack((A, -np.identity(m), np.zeros((m, m))))
    def DF(M, Y):
        DFr3 = np.hstack((np.zeros((m, n)), M, Y))
        return np.vstack((DFr1, DFr2, DFr3))
    
    k = 0
    dist = 100
    x, y, μ = startingPoint(Q, c, A, b, guess)
    
    while dist > tol and k < niter:
        ν = (y @ μ) / m
        e = np.ones(m)
        M = μ * np.eye(m)
        Y = y * np.eye(m)
        
        right = -F(x, y, μ) + np.concatenate((np.zeros(n + m), σ * ν * e))
        lu, piv = la.lu_factor(DF(M, Y))
        result = la.lu_solve((lu, piv), right)
        
        Δx = result[:n]
        Δy = result[n:m+n]
        Δμ = result[m+n:]
        
        # Maximum allowable step lengths
        τ = 0.95
        β_max = np.min([np.min(-μ[Δμ < 0] / Δμ[Δμ < 0]), 1.0])
        δ_max = np.min([np.min(-y[Δy < 0] / Δy[Δy < 0]), 1.0])

        β = np.min([1, τ * β_max])
        δ = np.min([1, τ * δ_max])
        α = np.min([β, δ])

        x_tilde = x + α * Δx
        y = y + α * Δy
        μ = μ + α * Δμ
        
        dist = la.norm(x_tilde - x)
        x = x_tilde
        k += 1
    return x

In [14]:
def startingPoint(G, c, A, b, guess):
    """
    Obtain an appropriate initial point for solving the QP
    .5x\trp Gx+x\trp cs.t.Ax>=b. Parameters:
        G -- symmetric positive semidefinite matrix shape (n,n)
        c -- array of length n
        A -- constraint matrix shape (m,n)
        b -- array of length m
        guess -- a tuple of arrays (x, y, l) of lengths n, m, and m, resp.
    Returns:
        a tuple of arrays (x0, y0, l0) of lengths n, m, and m, resp.
    """
    m, n = A.shape
    x0, y0, l0 = guess
    # initialize linear system
    N = np.zeros((n+m+m, n+m+m))
    N[:n,:n] = G
    N[:n, n+m:] = -A.T
    N[n:n+m, :n] = A
    N[n:n+m, n:n+m] = -np.eye(m)
    N[n+m:, n:n+m] = np.diag(l0)
    N[n+m:, n+m:] = np.diag(y0)
    rhs = np.empty(n+m+m)
    rhs[:n] = -(G.dot(x0) - A.T.dot(l0)+c)
    rhs[n:n+m] = -(A.dot(x0) - y0 - b)
    rhs[n+m:] = -(y0*l0)
    sol = la.solve(N, rhs)
    dx = sol[:n]
    dy = sol[n:n+m]
    dl = sol[n+m:]
    y0 = np.maximum(1, np.abs(y0 + dy))
    l0 = np.maximum(1, np.abs(l0+dl))
    return x0, y0, l0

In [15]:
# Objective function matrix Q, and vector c
Q = np.array([[1, -1], 
              [-1, 2]])
c = np.array([-2, -6])

# The constraint matrix A and vector b
A = np.array([[-1, -1], 
              [1, -2], 
              [-2, -1], 
              [1, 0], 
              [0, 1]])
b = np.array([-2, -2, -3, 0, 0])

m, n = A.shape
x = np.array([0.5, 0.5]) # Initial guess
y = np.ones(m)
μ = np.ones(m)
guess = (x, y, μ)

In [25]:
optimal_pt = qInteriorPoint(Q, c, A, b, guess)
print("The optimal point is:", optimal_pt)
print("The correct minimizer is [2/3, 4/3]")

The optimal point is: [0.66666667 1.33333333]
The correct minimizer is [2/3, 4/3]


Note that $[2/3, 4/3]=[0.66666667, 1.33333333]$

**Exercise 3**

In [6]:
def laplacian(n):
    """Construct the discrete Dirichlet energy matrix H for an n x n grid."""
    data = -1*np.ones((5, n**2))
    data[2,:] = 4
    data[1, n-1::n] = 0
    data[3, ::n] = 0
    diags = np.array([-n, -1, 0, 1, n])
    return spdiags(data, diags, n**2, n**2).toarray()

In [7]:
# Create the tent pole configuration.
L = np.zeros((n,n))
L[n//2-1:n//2+1,n//2-1:n//2+1] = .5
m = [n//6-1, n//6, int(5*(n/6.))-1, int(5*(n/6.))]
mask1, mask2 = np.meshgrid(m, m)
L[mask1, mask2] = .3
L = L.ravel()

# Set initial guesses.
x = np.ones((n,n)).ravel()
y = np.ones(n**2)
mu = np.ones(n**2)


In [8]:
n = 15
H = laplacian(n) # Grid side length n = 15
c = np.empty(n**2)
c.fill(-(n - 1) ** -2)
#A = ? 
# We defined A earlier, but what is the constraint matrix now?
# is it the same?

# Calculate the solution.
z = qInteriorPoint(H, c, A, L, (x,y,μ))[0].reshape((n,n))

# Plot the solution.
domain = np.arange(n)
X, Y = np.meshgrid(domain, domain)
fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')
ax1.plot_surface(X, Y, z, rstride=1, cstride=1, color='r')
plt.show()


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [10]:
# I am pretty sure the dimensions for H and c are right, but perhaps 
# the dimension of A is wrong. I did a few tests by changing
# the dimension of A but still got the same error

**Exercise 4**

In [26]:
# Cannot complete because portfolio.txt is not saved in the course repository.